# Webscraping boliga.dk

Importing packages

In [12]:
import requests
import time
import tqdm
import pandas as pd
import os
import json
import openpyxl

In [13]:
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import NaN
import seaborn as sns

In [14]:
from bs4 import BeautifulSoup

In [15]:
fp_data = Path.cwd() / "data" 
Path.mkdir(fp_data, exist_ok=True)  
print(fp_data)

C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data


In [23]:
def log(response, logfile, timestamp):
    with open(logfile, 'a') as log_file:
        log_file.write(f"{timestamp},{response.url},{response.status_code}\n")

csv_filename = fp_data /'html_data.csv'
log_success_filename = fp_data / 'log_success.csv'
log_failure_filename = fp_data / 'log_failure.csv'

if os.path.exists(csv_filename):
    existing_data = pd.read_csv(csv_filename, header=None)
    existing_htmls = existing_data[0].tolist()
else:
    existing_htmls = []

list_htmls = existing_htmls

links = []
for page in range(1,1330):
    url = f'https://www.boliga.dk/salg/resultater?searchTab=1&sort=date-d&saleType=1&zipcodeFrom=1000&zipcodeTo=2999&salesDateMin=2017&salesDateMax=2023&page={page}&propertyType=3'
    links.append(url)

workbook = openpyxl.Workbook()
worksheet = workbook.active

for url in tqdm.tqdm(links):
    try:
        response = requests.get(url, headers={'name':'Katrine Villadsen','email':'zlm357@econ.ku.dk'})
        response.raise_for_status()  # Check for HTTP error
    except requests.exceptions.RequestException as e:
        print(url)
        print(e)
        with open(log_failure_filename, 'a') as log_fail_file:
            log_fail_file.write(f"{url},{str(e)}\n")
        continue
    
    html = response.text
    list_htmls.append(html)
    current_time = time.strftime('%Y-%m-%d %H:%M:%S')
    log(response, log_success_filename, current_time)
    
    row_number = len(worksheet['A']) + 1
    worksheet.cell(row=row_number, column=1, value=html)

    pd.DataFrame(list_htmls).to_csv(csv_filename, index=False, header=False)

excel_filename = 'html_data.xlsx'
workbook.save(excel_filename)

print(f'HTML data saved to {excel_filename} and {csv_filename}')
print(f'Log of successful requests saved to {log_success_filename}')
print(f'Log of failed requests saved to {log_failure_filename}')

100%|████████████████████████████████████████████████████████████████████████████| 1329/1329 [2:43:50<00:00,  7.40s/it]


HTML data saved to html_data.xlsx and C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data\html_data.csv
Log of successful requests saved to C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data\log_success.csv
Log of failed requests saved to C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data\log_failure.csv


In [24]:
import pandas as pd
from bs4 import BeautifulSoup

# Parse data with BeautifulSoup
soup = BeautifulSoup(response.content,'lxml')

# Identify table to scrape by inspecting site
table_node = soup.find('app-sold-list-table') 

In [25]:
def parse_html_table(table_node):
    columns = []
    data = []
    
    thead = table_node.find('thead')
    if thead:
        columns_html = thead.find_all('th')
        columns = [col.text for col in columns_html]
    
    tbody = table_node.find('tbody')
    if tbody:
        rows_list = tbody.find_all('tr')
        for row_node in rows_list:
            row = [child.text for child in row_node.children]
            data.append(row)
    
    df = pd.DataFrame(data, columns=columns)
    return df

soup = BeautifulSoup(response.content, 'html.parser')
table_node = soup.find('table')

if table_node is not None:
    df = parse_html_table(table_node)
    print(df)
else:
    print("Table not found in the HTML content.")

                                 Boligtype & Adresse         Købesum   \
0    Ejerlejlighed EEjerlejlighed Oscar Pettifords...   3.600.000 kr.   
1    Ejerlejlighed EEjerlejlighed Helmer Søgårds A...   1.600.000 kr.   
2    Ejerlejlighed EEjerlejlighed Damhus Boulevard...   3.200.000 kr.   
3    Ejerlejlighed EEjerlejlighed Sallingvej 13, 2...   2.550.000 kr.   
4    Ejerlejlighed EEjerlejlighed Sankt Peders Str...   3.995.000 kr.   
5    Ejerlejlighed EEjerlejlighed Rødovrevej 301, ...     995.000 kr.   
6    Ejerlejlighed EEjerlejlighed Sorgenfrigade 7,...   2.295.000 kr.   
7    Ejerlejlighed EEjerlejlighed I.H.Mundts Vej 1...   2.095.000 kr.   
8    Ejerlejlighed EEjerlejlighed Bramslykkevej 8,...   1.625.000 kr.   
9    Ejerlejlighed EEjerlejlighed Kildebakkegårds ...   2.885.000 kr.   
10   Ejerlejlighed EEjerlejlighed Ahornsgade 25, 3...   3.599.000 kr.   
11   Ejerlejlighed EEjerlejlighed Weidekampsgade 6...   3.300.000 kr.   
12   Ejerlejlighed EEjerlejlighed Brønshøj Kirkeve.

In [26]:
def parse_html_table(table_node):
    columns_html = table_node.thead.find_all('th')
    columns = [col.text for col in columns_html]

    rows_list = table_node.tbody.find_all('tr')

    data = []
    for row_node in rows_list:
        row = []
        for child in row_node.children:
            row.append(child.text)
        data.append(row)
    df = pd.DataFrame(data,columns=columns)
    return df

df = parse_html_table(table_node)
df

,Boligtype & Adresse,Købesum,Dato & Type,m² & Kr. / m²,Vær.,Byggeår,Den procentuelle forskel mellem seneste udbudspris og salgsprisen %,
0,Ejerlejlighed EEjerlejlighed Oscar Pettifords...,3.600.000 kr.,08-01-2017Alm. Salg,92 m² 39.130 kr/m²,3,2008,0%,Aktuel værdi
1,Ejerlejlighed EEjerlejlighed Helmer Søgårds A...,1.600.000 kr.,07-01-2017Alm. Salg,60 m² 26.667 kr/m²,2,1950,-3%,Aktuel værdi
2,Ejerlejlighed EEjerlejlighed Damhus Boulevard...,3.200.000 kr.,07-01-2017Alm. Salg,109 m² 18.391 kr/m²,6,1963,,Aktuel værdi
3,"Ejerlejlighed EEjerlejlighed Sallingvej 13, 2...",2.550.000 kr.,07-01-2017Alm. Salg,99 m² 25.758 kr/m²,4,1932,-2%,Aktuel værdi
4,Ejerlejlighed EEjerlejlighed Sankt Peders Str...,3.995.000 kr.,07-01-2017Alm. Salg,106 m² 37.689 kr/m²,3,1809,,Aktuel værdi
5,"Ejerlejlighed EEjerlejlighed Rødovrevej 301, ...",995.000 kr.,07-01-2017Alm. Salg,45 m² 22.111 kr/m²,1,1967,,Aktuel værdi
6,"Ejerlejlighed EEjerlejlighed Sorgenfrigade 7,...",2.295.000 kr.,07-01-2017Alm. Salg,62 m² 37.016 kr/m²,2,1931,,Aktuel værdi
7,Ejerlejlighed EEjerlejlighed I.H.Mundts Vej 1...,2.095.000 kr.,07-01-2017Alm. Salg,75 m² 27.933 kr/m²,3,1938,,Aktuel værdi
8,"Ejerlejlighed EEjerlejlighed Bramslykkevej 8,...",1.625.000 kr.,07-01-2017Alm. Salg,54 m² 30.093 kr/m²,2,1932,-1%,Aktuel værdi
9,Ejerlejlighed EEjerlejlighed Kildebakkegårds ...,2.885.000 kr.,07-01-2017Alm. Salg,100 m² 28.850 kr/m²,4,1952,,Aktuel værdi


In [27]:
all_data = pd.DataFrame()

for url in tqdm.tqdm(links):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP error
    except requests.exceptions.RequestException as e:
        print(url)
        print(e)
        with open(log_failure_filename, 'a') as log_fail_file:
            log_fail_file.write(f"{url},{str(e)}\n")
        continue

    soup = BeautifulSoup(response.content, 'lxml')
    # Identify the table to scrape by inspecting the site
    table_node = soup.find('app-sold-list-table')
    
    if table_node is not None:
        # Parse the table data using the provided function
        df = parse_html_table(table_node)
        # Append the parsed data to the overall DataFrame
        all_data = all_data.append(df, ignore_index=True)
    else:
        print("Table not found in the HTML content.")

    current_time = time.strftime('%Y-%m-%d %H:%M:%S')
    log(response, log_success_filename, current_time)

# Save the DataFrame to a CSV file
all_data.to_csv(csv_filename, index=False)
print(f'HTML data saved to {csv_filename}')
print(f'Log of successful requests saved to {log_success_filename}')
print(f'Log of failed requests saved to {log_failure_filename}')

  0%|                                                                                         | 0/1329 [00:00<?, ?it/s]C:\Users\Bruger\AppData\Local\Temp\ipykernel_10704\1606127766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df, ignore_index=True)
  0%|                                                                                 | 1/1329 [00:01<30:30,  1.38s/it]C:\Users\Bruger\AppData\Local\Temp\ipykernel_10704\1606127766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df, ignore_index=True)
  0%|                                                                                 | 2/1329 [00:02<27:49,  1.26s/it]C:\Users\Bruger\AppData\Local\Temp\ipykernel_10704\1606127766.py:25: FutureWarning: The frame.append method is deprecated and will be r

HTML data saved to C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data\html_data.csv
Log of successful requests saved to C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data\log_success.csv
Log of failed requests saved to C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data\log_failure.csv


In [28]:
display(all_data)

csv_filename = fp_data / 'combined_data.csv'
all_data.to_csv(csv_filename, index=False)
print(f'DataFrame saved to {csv_filename}')

,Boligtype & Adresse,Købesum,Dato & Type,m² & Kr. / m²,Vær.,Byggeår,Den procentuelle forskel mellem seneste udbudspris og salgsprisen %,
0,Ejerlejlighed EEjerlejlighed Søndre Fasanvej ...,2.650.000 kr.,02-08-2023Alm. Salg,67 m² 39.552 kr/m²,1,1975,-4%,Aktuel værdi
1,"Ejerlejlighed EEjerlejlighed Herlev Bygade 2,...",1.153.000 kr.,02-08-2023Alm. Salg,49 m² 23.531 kr/m²,1,1962,,Aktuel værdi
2,Ejerlejlighed EEjerlejlighed Christoffers All...,1.175.000 kr.,02-08-2023Alm. Salg,42 m² 27.976 kr/m²,2,1943,,Aktuel værdi
3,Ejerlejlighed EEjerlejlighed Gyngemose Parkve...,3.451.000 kr.,02-08-2023Alm. Salg,97 m² 35.577 kr/m²,4,2007,-4%,Aktuel værdi
4,Ejerlejlighed EEjerlejlighed Strandlodsvej 23...,5.550.000 kr.,02-08-2023Alm. Salg,99 m² 56.061 kr/m²,3,2021,-3%,Aktuel værdi
...,...,...,...,...,...,...,...,...
66445,"Ejerlejlighed EEjerlejlighed Saksen C 4, st. ...",1.600.000 kr.,04-01-2017Alm. Salg,116 m² 13.793 kr/m²,4,1972,,Aktuel værdi
66446,"Ejerlejlighed EEjerlejlighed Engvej 40, st. t...",3.275.000 kr.,04-01-2017Alm. Salg,105 m² 31.190 kr/m²,4,2000,-2%,Aktuel værdi
66447,"Ejerlejlighed EEjerlejlighed Hamletsgade 4, 4...",1.458.000 kr.,04-01-2017Alm. Salg,58 m² 25.138 kr/m²,2,1975,1%,Aktuel værdi
66448,"Ejerlejlighed EEjerlejlighed Schubertsvej 6, ...",2.600.000 kr.,04-01-2017Alm. Salg,74 m² 35.135 kr/m²,3,1932,,Aktuel værdi


DataFrame saved to C:\Users\Bruger\OneDrive\Dokumenter\GitHub\isds2023\assignments\Exam project\data\combined_data.csv
